In [169]:
import pandapower as pp
import pandas as pd
import numpy as np
from pandapower.timeseries.output_writer import OutputWriter


# Install pandapower in a new environment by pip install pandapower==2.13.1
net = pp.from_pickle('./aggregated_grid_2018_with_generators_loads_costs_controllers.p')

out_path = "./AC_OPF"

line_data = pd.read_csv("./original_lineInputData.csv")
# ow = OutputWriter(net, output_path=out_path, output_file_type=".csv", csv_separator=",")
# ow.remove_log_variable("res_gen", "vm_pu")

In [170]:
# GET MAX S_MVA PER INDIVIDUAL LINE

line_max_i_ka = net.line.max_i_ka
line_v_nominal_kv = net.line.type.apply(lambda x: float(x[:-2])) # Don't include unit

line_max_s_mva = line_max_i_ka.mul(line_v_nominal_kv)

line_max_s_mva


0      880.00
1      880.00
2      550.00
3      550.00
4      550.00
5      550.00
6      550.00
7      550.00
8      550.00
9      550.00
10     506.00
11     506.00
12     506.00
13     506.00
14     506.00
15     550.00
16     550.00
17     506.00
18     506.00
19     550.00
20     550.00
21     950.00
22     950.00
23     950.00
24    1098.20
25    1098.20
26     570.00
27     570.00
28    1140.00
29    1140.00
30    1520.00
31    1520.00
32     950.00
33     950.00
34     950.00
35     950.00
36     950.00
37     950.00
38    1520.00
39    1520.00
40    1520.00
41    1520.00
42     950.00
43     950.00
44    1520.00
45    1520.00
46     950.00
47     950.00
48     950.00
49     950.00
50     950.00
51     950.00
52     950.00
53     950.00
54    1520.00
55    1520.00
56    1073.88
57    1073.88
58     950.00
59    1520.00
60    1520.00
61     950.00
62     950.00
63    1073.88
64    1073.88
65     988.00
66     988.00
67     577.60
68     577.60
69    1520.00
70    1520.00
71    

In [171]:
# GET MAX S_MVA FOR TRANSFORMER LINES

trafo_loading_percent = net.trafo.max_loading_percent.div(100) # Loading percentage
trafo_sn_mva = net.trafo.sn_mva # Nominal power

trafo_max_s_mva = trafo_loading_percent.mul(trafo_sn_mva)

# trafo_max_s_mva

In [172]:
# GET MAX S_MVA PER LINE

slack_bus = 35
swap_slack_bus = {0: slack_bus, slack_bus: 0} # Swap slack bus 0 and 35

from_bus = pd.concat([net.line.from_bus, net.trafo.hv_bus] , ignore_index=True) # Appending transformer buses
from_bus = from_bus.replace(swap_slack_bus) # Swap slack bus

to_bus = pd.concat([net.line.to_bus, net.trafo.lv_bus], ignore_index=True)
to_bus = to_bus.replace(swap_slack_bus)

line_max_s_mva = pd.concat([line_max_s_mva, trafo_max_s_mva], ignore_index=True) # Append transformer limits

indv_line = pd.DataFrame({"From Bus": from_bus, "To Bus": to_bus, "Max MVA": line_max_s_mva}) # Individual lines
line = indv_line.groupby(["From Bus", "To Bus"], as_index=False)["Max MVA"].sum() # Add up parallel lines
line[["From Bus", "To Bus"]] += 1 # Correct bus index
line

,From Bus,To Bus,Max MVA
0,1,29,3040.00
1,3,2,1100.00
2,4,6,1100.00
3,4,8,1100.00
4,6,2,1100.00
5,7,9,1012.00
6,8,7,1012.00
7,8,10,1100.00
8,8,36,506.00
9,9,5,1012.00


In [173]:
# ADD USER GENERATORS

max_user_1 = line["Max MVA"].max() # Max line loading for first user generator
max_user_2 = line["Max MVA"].max()

line.loc[len(line)] = [13, 46, max_user_1]
line.loc[len(line)] = [4, 47, max_user_2]
line

,From Bus,To Bus,Max MVA
0,1.0,29.0,3040.00
1,3.0,2.0,1100.00
2,4.0,6.0,1100.00
3,4.0,8.0,1100.00
4,6.0,2.0,1100.00
5,7.0,9.0,1012.00
6,8.0,7.0,1012.00
7,8.0,10.0,1100.00
8,8.0,36.0,506.00
9,9.0,5.0,1012.00


In [174]:
# Transform to p.u.

s_base = 100.0 # MVA

line["Max MVA"] /= s_base
line

,From Bus,To Bus,Max MVA
0,1.0,29.0,30.4000
1,3.0,2.0,11.0000
2,4.0,6.0,11.0000
3,4.0,8.0,11.0000
4,6.0,2.0,11.0000
5,7.0,9.0,10.1200
6,8.0,7.0,10.1200
7,8.0,10.0,11.0000
8,8.0,36.0,5.0600
9,9.0,5.0,10.1200


In [176]:
line_data[["From Bus", "To Bus", "R", "X", "G", "B", "Max MVA"]]

,From Bus,To Bus,R,X,G,B,Max MVA
0,44,27,5.536000e-05,0.000644,0,0.073548,16.00
1,3,2,7.737170e-04,0.008534,0,0.095456,16.00
2,4,8,1.067711e-03,0.011757,0,0.131024,16.00
3,4,6,7.034730e-04,0.007736,0,0.086640,16.00
4,4,47,7.737170e-04,0.002858,0,0.095456,16.00
5,6,2,1.081551e-03,0.011895,0,0.133152,16.00
6,7,9,4.514210e-04,0.006541,0,0.077824,16.00
7,8,36,2.045340e-03,0.023637,0,0.060800,8.00
8,8,7,1.019571e-03,0.011792,0,0.121296,16.00
9,8,10,4.132000e-04,0.007200,0,0.069008,16.00


In [177]:
line_data = line_data[["From Bus", "To Bus", "R", "X", "G", "B", "Max MVA"]]
merged = pd.merge(line_data, line, on=["From Bus", "To Bus"], suffixes=("", "_new")) # Merge dataframes where From Bus and To Bus are same
line_data["Max MVA"] = merged["Max MVA_new"]
line_data

,From Bus,To Bus,R,X,G,B,Max MVA
0,44,27,5.536000e-05,0.000644,0,0.073548,19.0000
1,3,2,7.737170e-04,0.008534,0,0.095456,11.0000
2,4,8,1.067711e-03,0.011757,0,0.131024,11.0000
3,4,6,7.034730e-04,0.007736,0,0.086640,11.0000
4,4,47,7.737170e-04,0.002858,0,0.095456,30.4000
5,6,2,1.081551e-03,0.011895,0,0.133152,11.0000
6,7,9,4.514210e-04,0.006541,0,0.077824,10.1200
7,8,36,2.045340e-03,0.023637,0,0.060800,5.0600
8,8,7,1.019571e-03,0.011792,0,0.121296,10.1200
9,8,10,4.132000e-04,0.007200,0,0.069008,11.0000


In [178]:
line_data.to_csv(out_path + "/lineInputData.csv", index=False)